## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [15]:
class Node:
    def __init__(self, weight: int, chars: str):
        self.weight = weight
        self.chars = chars

    def __lt__(self, other):
        return self.weight < other.weight
    
    def __add__(self, other):
        return Node(self.weight + other.weight, self.chars + other.chars)
    
    def __iter__(self):
        return iter(self.chars)

class Huffman:
    @staticmethod
    def count_frequency(text: str) -> dict[str, int]:
        frequency = {}

        for char in text:
            frequency.setdefault(char, 0)
            frequency[char] += 1

        return frequency


    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        if not text:
            return '', {}

        frequency = self.count_frequency(text)

        tree = [Node(weight, [[char,'']]) for char, weight in frequency.items()]

        if len(tree) == 1:
            return '0'*frequency[text[0]], {text[0]: '0'}

        while len(tree) > 1:
            first_min=tree.pop(tree.index(min(tree)))
            second_min=tree.pop(tree.index(min(tree)))
            for char in first_min:
                char[1] = '1' + char[1]
            for char in second_min:
                char[1] = '0' + char[1]
            tree.append(first_min+second_min)

        huffman_code = {item[0]: item[1] for item in tree[0]}

        encoded_text = ''.join(huffman_code[char] for char in text)

        return encoded_text, huffman_code


    def decode(self, code: str, coding_dict: dict[str, str]) -> str:
        reverse=dict((v,k) for k,v in coding_dict.items())

        decoded_text = ""
        temp = ""

        for char in code:
            temp += char
            if temp in reverse:
                decoded_text += reverse[temp]
                temp = ""

        return decoded_text
    

if __name__ == "__main__":
    huffman = Huffman()
    text = "abracadabra"
    encoded_text, coding_dict = huffman.encode(text)
    print(f"Original text: {text}")
    print(f"Encoded text: {encoded_text}")
    print(f"Coding dictionary: {coding_dict}")
    print(f"Decoded text: {huffman.decode(encoded_text, coding_dict)}")
    print(f'Bit length of original text: {len(text) * 8}')
    print(f'Bit length of encoded text: {len(encoded_text)}')
    print(f'Ratio of encoded text to original text: {len(encoded_text) / (len(text) * 8)}')
    assert huffman.decode(encoded_text, coding_dict) == text

Original text: abracadabra
Encoded text: 10010001011101010010001
Coding dictionary: {'a': '1', 'c': '011', 'd': '010', 'b': '001', 'r': '000'}
Decoded text: abracadabra
Bit length of original text: 88
Bit length of encoded text: 23
Ratio of encoded text to original text: 0.26136363636363635


# Алгоритм LZW

In [33]:
class LZW:
    def encode(self, text: str) -> tuple[str, list]:
        if not text:
            return '', []

        dictionary={}
        i=0

        for char in text:
            if len(dictionary)==len(set(text)):
                break
            if char not in dictionary:
                dictionary[char]=i
                i+=1
        
        result_dictionary = list(dictionary.keys())
 
        encoded_text = ''
        temp = ""

        for char in text:
            temp += char
            if temp not in dictionary:
                encoded_text+=(str(dictionary[temp[:-1]]))+' '
                dictionary[temp] = len(dictionary)
                temp = char

        encoded_text+=str(dictionary[temp])

        return encoded_text, result_dictionary

    def decode(self, code: str, coding_dict: list) -> str:
        coding_dict = {i : char for i, char in enumerate(coding_dict)}
        coding_dict_start = coding_dict.copy()

        decoded_text = ""
        temp=''
        code = list(map(int,code.split()))

        for num in code:
            if num in coding_dict:
                decoded_text += coding_dict[num]
                temp+=coding_dict[num]
                if num in coding_dict_start:
                    if temp not in coding_dict.values():
                        coding_dict[len(coding_dict)] = temp
                        temp = temp[-1]
                else:
                    temp_2=''
                    for char in temp:
                        temp_2+=char
                        if temp_2 not in coding_dict.values():
                            coding_dict[len(coding_dict)] = temp_2
                            temp_2 = temp_2[:-1]
                            break
                    temp = temp[len(temp_2):]
            else:
                temp=temp+temp[0]
                coding_dict[len(coding_dict)] = temp
                decoded_text += coding_dict[num]
        return decoded_text


if __name__ == "__main__":
    lzw = LZW()
    text = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,'
    encoded_text, coding_dict = lzw.encode(text)
    print(f"Original text: {text}")
    print(f"Encoded text: {encoded_text}")
    print(f"Coding dictionary: {coding_dict}")
    print(f"Decoded text: {lzw.decode(encoded_text, coding_dict)}")
    print(f'Bit length of original text: {len(text) * 8}')
    print(f'Bit length of encoded text: {len(encoded_text)*8}')
    print(f'Ratio of encoded text to original text: {len(encoded_text)*8 / (len(text)*8)}')
    assert lzw.decode(encoded_text, coding_dict) == text

Original text: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nullam non nisl nec nunc fermentum fermentum. Nullam nec tellus fermentum,Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam ac ante at ipsum fermentum fermentum. Nul

# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [31]:
from math import log2
from random import randint
import time

class LZ77:
    def __init__(self, buffer_size: int) -> None:
        self.buffer_size = buffer_size

    def encode(self, text: str) -> str:
        i = 0
        code = []
        text_len = len(text)
        while i < text_len:
            buffer_start = max(0, i - self.buffer_size)
            buffer_end = i
            code_len = 0
            last_code_len = 0
            first_i = 0
            for buffer_i in range(buffer_start, buffer_end):
                code_len = 0
                encoded_el = ""
                if text[buffer_i] == text[i]:
                    code_len += 1
                    encoded_el = text[buffer_i]
                    buffer_i2 = buffer_i + 1
                    while buffer_i2 < buffer_end + code_len:
                        if i + code_len >= text_len:
                            break
                        if text[buffer_i2] == text[i + code_len]:
                            code_len += 1
                            encoded_el += text[buffer_i2]
                        else:
                            break
                        buffer_i2 += 1
                if code_len >= last_code_len:
                    first_i = i - buffer_i if code_len else 0
                    last_code_len = code_len
            last_el = text[i + last_code_len] if i + last_code_len < text_len else ""
            code.append((first_i,last_code_len,last_el))
            i += last_code_len + 1
        return self._list2text(code)

    @staticmethod
    def _text2list(text: str) -> list[tuple]:
        code = []
        while text:
            text = text.split(",", 2)
            code.append((int(text[0]), int(text[1]), text[2][0] if text[2] else ""))
            text = text[2][1:] if text[2] else ""
        return code

    @staticmethod
    def _list2text(lst: list[tuple]) -> str:
        return "".join(map(lambda el: f"{el[0]},{el[1]},{el[2]}", lst))

    def decode(self, text: str) -> str:
        i = 0
        code = self._text2list(text)
        decoded_text = ""
        for buffer_i, code_len, last_el in code:
            if code_len - buffer_i > 0:
                encoded_part = decoded_text[i - buffer_i: i] * (code_len // buffer_i + 1)
                decoded_text += encoded_part[:code_len] + last_el
            else:
                decoded_text += decoded_text[i - buffer_i: i - buffer_i + code_len] + last_el
            i += code_len + 1
        return decoded_text


if __name__ == "__main__":
    alphabet = "abcdefghigklmnopqrstuvwxyz., '1234567890()"
    random_text = "".join(alphabet[randint(0, len(alphabet) - 1)] for _ in range(1000))
    # random_text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
    # random_text = "abacabacabadaca"
    buffer_size = len(random_text)
    lz = LZ77(buffer_size)
    encoded_text = lz.encode(random_text)
    encoded_text = lz.encode(random_text)
    print(f"Original text: {random_text}")
    print(f"Encoded text: {encoded_text}")
    print(f"Decoded text: {lz.decode(encoded_text)}")
    print(f'Bit length of original text: {len(random_text) * 8}')
    encoded_len = 0
    for els in lz._text2list(encoded_text):
        for sym in els:
            if isinstance(sym, int):
                encoded_len += len(bin(sym)[2:])
            else:
                encoded_len += 8
    print(f'Ratio of encoded text to original text: {encoded_len / (len(random_text) * 8)}')
    assert lz.decode(encoded_text) == random_text

Original text: ty7zzw26i0w.nqm.(csgykrl,9vif o7xq0'21d)'bnk 84y6l'fodb4xfn34 9w7guq wo6(wp27(hsg(dkei,2dmw4(ginhrwodgkuy5k4.7rgtrp,h s9l0esip3h5tea.tw(209smei'klm'3tgkeaz,k(o.lglm)wr q3x,3e(zu2k678593slp4(opys iue1sfnulzglt185k)d wgqdww71g51fih(uv8qi0ly570y24hxal933aczz.mug38m60a uvmz0q40tt),94,twakaikkp.lh0lv9d 4,'ls6a'acp5aqhyq47mh3irprt7dt)iffwz9cf ga0vy31su 30xnqexfw5vu0m,8t,87ggfw57ie4ys3p2h8wbfb,a7ai2u46x1,38ep'3l9 ,8huzd6i,ypt(4mgkb4mbcicyie 4p3uvtxxp2.,dbz8r'eses4vgb(4v6h(i,t9enkw,5uf1ng9utbbnl4uai7,812e9aqyoyqwq.5w,8.abkvpabsmrc05z0,f.vpgi2l04,ghmeuigfz1nxzq.)5a1a1n 19(rl2it3xz73fv6wzzla1q14k5e t)9g5'f)9(lq8x421i7m4r0p437w9yxgb.gp2tywk6uuufv.rsif5 dorrrh88fki(6zz(,as)ccd325k5c,wg8 .oe0.s4bpgb,s44s2h218egc.,))y6dx8abah'0mby9b)8x zrw,47zbk 1f7g2og,3mx's'xex, .o,goahz79 25gd7dgt(u,fev0g7m.b,44azyg9kvr5t0sbdlh,4gab4ryg0qz.y(6r1lk3,x4wbo(ie0ld,.0f4.f4i2zrn4vnvwzfl9pw30f3bk0'2l(''h2,)dv9bwlt9x0'x84z,7e.8xvip)3z)e1xmg.kd7n1zy9mg34 ngw.ody9rwsyw3px934w g45aildknf1d0yuqxgaapg3dpn(998wv

# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size: int):
        pass

    def encode(self, text: str) -> str:
        pass

    def decode(self, code: str) -> str:
        pass